In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# ==========================================
# 1. Configuration & ID Definitions
# ==========================================
raw_dir = r"C:\AINutriCare\Data\Raw"
output_dir = r"C:\AINutriCare\Data\Transformed"
os.makedirs(output_dir, exist_ok=True)

def get_raw_path(filename): return os.path.join(raw_dir, filename)

# Mapping User List to MIMIC-III Item IDs
CONFIG = {
    # --- LABS ---
    "Labs": {
        "Glucose": [50931, 50809],
        "Creatinine": [50912],
        "BUN": [51006],
        "Sodium": [50983, 50824],
        "Potassium": [50971, 50822],
        "Hemoglobin": [51222, 50811],
        "WBC": [51301, 51300],
        "Lactate": [50813]
    },
    # --- VITALS ---
    "Vitals": {
        "Heart Rate": [211, 220045],
        # MAP IDs (Mean Arterial Pressure)
        "MAP": [456, 52, 220181, 220052], 
        "Respiratory Rate": [618, 615, 220210, 224690],
        "Temperature": [223761, 678], # Fahrenheit
    },
    # --- MEDS (Inputs) ---
    "Inputs": {
        "Vasopressors": [221906, 221289, 221662, 222315], # Norepi, Vasopressin...
        "Sedatives": [222168, 221668, 221744], # Propofol, Midazolam...
        "Antibiotics": [225798, 225837, 225840, 225842, 225843], # Vancomycin...
        "Insulin": [223258, 223260, 223262],
    }
}

LAB_IDS = [id for sublist in CONFIG["Labs"].values() for id in sublist]
VITAL_IDS = [id for sublist in CONFIG["Vitals"].values() for id in sublist]
MED_IDS = [id for sublist in CONFIG["Inputs"].values() for id in sublist]

# ==========================================
# 2. Helper Function (Time-Series Pivot)
# ==========================================
def process_time_series(df, time_col, val_col, item_col, id_map, aggregation='mean'):
    # Merge with Admissions to get 'Admit Time'
    df = pd.merge(df, admissions[['hadm_id', 'admittime']], on='hadm_id', how='inner')
    
    # Calculate 'Hours Since Admission'
    df['hours_in'] = (df[time_col] - df['admittime']).dt.total_seconds() / 3600
    
    # Filter: Keep only Hours 0 to 24
    df_24 = df[(df['hours_in'] >= 0) & (df['hours_in'] <= 24)].copy()
    df_24['hour_bin'] = np.floor(df_24['hours_in']).astype(int)
    
    # Map ItemIDs to Labels (e.g., 211 -> "Heart Rate")
    reverse_map = {}
    for label, ids in id_map.items():
        for i in ids: reverse_map[i] = label
    df_24['feature_label'] = df_24[item_col].map(reverse_map)
    
    # Pivot: Rows=Patient/Hour, Cols=Feature
    pivot = df_24.pivot_table(
        index=['hadm_id', 'hour_bin'], 
        columns='feature_label', 
        values=val_col, 
        aggfunc=aggregation
    )
    return pivot

# ==========================================
# 3. Main ETL Logic
# ==========================================
try:
    print("Loading Admissions...")
    admissions = pd.read_csv(get_raw_path('admissions.csv'), parse_dates=['admittime'])

    # --- PROCESS LABS ---
    print("Processing Labs...")
    labevents = pd.read_csv(get_raw_path('labevents.csv'), parse_dates=['charttime'])
    lab_subset = labevents[labevents['itemid'].isin(LAB_IDS)].copy()
    lab_hourly = process_time_series(lab_subset, 'charttime', 'valuenum', 'itemid', CONFIG["Labs"], 'mean')

    # --- PROCESS VITALS (MAP, HR, etc.) ---
    print("Processing Vitals...")
    chartevents = pd.read_csv(get_raw_path('chartevents.csv'), parse_dates=['charttime'])
    vitals_subset = chartevents[chartevents['itemid'].isin(VITAL_IDS)].copy()
    vitals_hourly = process_time_series(vitals_subset, 'charttime', 'valuenum', 'itemid', CONFIG["Vitals"], 'mean')

    # --- PROCESS MEDS & FLUID INPUT ---
    print("Processing Inputs...")
    try:
        inputevents = pd.read_csv(get_raw_path('inputevents_mv.csv'), parse_dates=['starttime'])
        
        # 1. Meds (Binary or Amount)
        meds_subset = inputevents[inputevents['itemid'].isin(MED_IDS)].copy()
        meds_hourly = process_time_series(meds_subset, 'starttime', 'amount', 'itemid', CONFIG["Inputs"], 'sum')
        
        # 2. Total Input Volume (for Fluid Balance)
        inputevents = pd.merge(inputevents, admissions[['hadm_id', 'admittime']], on='hadm_id', how='inner')
        inputevents['hours_in'] = (inputevents['starttime'] - inputevents['admittime']).dt.total_seconds() / 3600
        input_24 = inputevents[(inputevents['hours_in'] >= 0) & (inputevents['hours_in'] <= 24)].copy()
        input_24['hour_bin'] = np.floor(input_24['hours_in']).astype(int)
        
        total_input = input_24.pivot_table(
            index=['hadm_id', 'hour_bin'], values='amount', aggfunc='sum'
        ).rename(columns={'amount': 'Input'})
        
    except FileNotFoundError:
        print("Warning: inputevents_mv.csv not found. Meds/Input set to 0.")
        meds_hourly = pd.DataFrame()
        total_input = pd.DataFrame()

    # --- PROCESS FLUID OUTPUT ---
    print("Processing Outputs...")
    try:
        outputevents = pd.read_csv(get_raw_path('outputevents.csv'), parse_dates=['charttime'])
        outputevents = pd.merge(outputevents, admissions[['hadm_id', 'admittime']], on='hadm_id', how='inner')
        outputevents['hours_in'] = (outputevents['charttime'] - outputevents['admittime']).dt.total_seconds() / 3600
        output_24 = outputevents[(outputevents['hours_in'] >= 0) & (outputevents['hours_in'] <= 24)].copy()
        output_24['hour_bin'] = np.floor(output_24['hours_in']).astype(int)
        
        total_output = output_24.pivot_table(
            index=['hadm_id', 'hour_bin'], values='value', aggfunc='sum'
        ).rename(columns={'value': 'Output'})
        
    except FileNotFoundError:
        print("Warning: outputevents.csv not found. Output set to 0.")
        total_output = pd.DataFrame()

    # ==========================================
    # 4. Merging & Final Calculations
    # ==========================================
    print("Merging Features...")
    combined_df = pd.concat([lab_hourly, vitals_hourly, meds_hourly, total_input, total_output], axis=1)

    # Reindex to ensure every patient has exactly 24 rows
    unique_stays = combined_df.index.get_level_values(0).unique()
    full_idx = pd.MultiIndex.from_product([unique_stays, range(24)], names=['hadm_id', 'hour_bin'])
    combined_df = combined_df.reindex(full_idx)

    # Handle Missing 'Input'/'Output' before calculation
    if 'Input' not in combined_df.columns: combined_df['Input'] = 0
    if 'Output' not in combined_df.columns: combined_df['Output'] = 0
    combined_df['Input'] = combined_df['Input'].fillna(0)
    combined_df['Output'] = combined_df['Output'].fillna(0)

    # --- CALCULATE FLUID BALANCE ---
    combined_df['Fluid Balance'] = combined_df['Input'] - combined_df['Output']

    # Clean Up: Drop raw Input/Output if you only want Balance (Optional - keeping them doesn't hurt)
    # The user asked for "Fluid Balance", so we focus on that.

    # Impute Missing Values
    # Forward Fill (e.g., Creatinine doesn't change every hour)
    combined_df = combined_df.groupby('hadm_id').ffill()
    
    # Fill Meds with 0 (No record = drug not given)
    for col in CONFIG["Inputs"].keys():
        if col in combined_df.columns: combined_df[col] = combined_df[col].fillna(0)
            
    # Fill remaining Vitals/Labs with 0 or Mean (using 0 for safety in deep learning, scaler fixes it later)
    combined_df = combined_df.fillna(0)

    # --- FINAL COLUMN SELECTION ---
    # Ensure columns appear exactly as requested
    requested_cols = [
        "Heart Rate", "MAP", "Respiratory Rate", "Temperature", 
        "Glucose", "Creatinine", "BUN", "Sodium", "Potassium", "Hemoglobin", "WBC", "Lactate",
        "Fluid Balance", 
        "Vasopressors", "Sedatives", "Antibiotics", "Insulin"
    ]
    
    # Add missing columns as 0
    for col in requested_cols:
        if col not in combined_df.columns: combined_df[col] = 0
            
    final_df = combined_df[requested_cols]
    final_df.reset_index(inplace=True)

    # Save
    out_file = os.path.join(output_dir, 'processed_mimic_24h_final.csv')
    final_df.to_csv(out_file, index=False)
    
    print("\nSUCCESS!")
    print(f"Features: {len(requested_cols)}")
    print(f"Saved to: {out_file}")
    print(final_df.head())

except Exception as e:
    print(f"Error: {e}")

Loading Admissions...
Processing Labs...
Processing Vitals...
Processing Inputs...
Processing Outputs...
Merging Features...

SUCCESS!
Features: 17
Saved to: C:\AINutriCare\Data\Transformed\processed_mimic_24h_final.csv
      hadm_id  hour_bin  Heart Rate  MAP  Respiratory Rate  Temperature  \
0  20044587.0         0         0.0  0.0               0.0          0.0   
1  20044587.0         1         0.0  0.0               0.0          0.0   
2  20044587.0         2         0.0  0.0               0.0          0.0   
3  20044587.0         3         0.0  0.0               0.0          0.0   
4  20044587.0         4         0.0  0.0               0.0          0.0   

   Glucose  Creatinine  BUN  Sodium  Potassium  Hemoglobin  WBC  Lactate  \
0    108.0         0.0  0.0   142.0        3.6        12.0  0.0      1.4   
1    141.0         0.0  0.0   142.0        5.2        10.2  0.0      2.7   
2    171.0         0.0  0.0   142.0        5.1         9.5  0.0      1.8   
3    171.0         0.0  0

In [3]:
# ==========================================
# 1. Configuration & Imports
# ==========================================
raw_dir = r"C:\AINutriCare\Data\Raw"
processed_dir = r"C:\AINutriCare\Data\Transformed"

def get_raw_path(filename): return os.path.join(raw_dir, filename)
def get_proc_path(filename): return os.path.join(processed_dir, filename)

# ==========================================
# 2-4. Load Admissions & Inspect Labels
# ==========================================
print("Step 1: Loading Admissions Data...")
try:
    admissions = pd.read_csv(get_raw_path('admissions.csv'))
    
    # Keep only what we need: Patient ID and the Target Label (Death Flag)
    # hospital_expire_flag: 1 = Died in Hospital, 0 = Survived
    admission_labels = admissions[['hadm_id', 'hospital_expire_flag']].set_index('hadm_id')
    
    print(f"Admissions Loaded. Total Records: {len(admission_labels)}")
    print(admission_labels.head())

except FileNotFoundError:
    print("Error: admissions.csv not found.")
    exit()

# ==========================================
# 5. Check Label Distribution
# ==========================================
print("\nStep 2: Checking Class Balance...")
counts = admission_labels['hospital_expire_flag'].value_counts()
print(f"Survived (0): {counts.get(0, 0)}")
print(f"Expired  (1): {counts.get(1, 0)}")
print(f"Imbalance Ratio: 1:{counts.get(0, 0) // counts.get(1, 1)}")
# Note: ICU data is usually heavily imbalanced (mostly survivors). 
# We will handle this during training (Milestone 3) using Class Weights.

# ==========================================
# 6. Load Final Time-Series Dataset
# ==========================================
print("\nStep 3: Loading Processed Time-Series Data...")
try:
    # Load the file generated in the previous step
    df = pd.read_csv(get_proc_path('processed_mimic_24h_final.csv'))
    print(f"Time-Series Shape: {df.shape}")
    
    # Ensure data is sorted by Patient and Time (Crucial for LSTM)
    df = df.sort_values(by=['hadm_id', 'hour_bin'])
    
except FileNotFoundError:
    print("Error: Processed CSV not found. Run the previous data processing step first.")
    exit()

# ==========================================
# 7 & 8. Create X (3D Features) and y (Labels)
# ==========================================
print("\nStep 4: Aligning X and y...")

# Get list of unique patients in our time-series data
unique_patients = df['hadm_id'].unique()
num_patients = len(unique_patients)
num_hours = 24
# Drop metadata to count features
feature_cols = [c for c in df.columns if c not in ['hadm_id', 'hour_bin', 'index', 'level_0']]
num_features = len(feature_cols)

print(f"Dimensions -> Patients: {num_patients}, Hours: {num_hours}, Features: {num_features}")

# --- Construct X (Features) ---
# Reshape (Rows) -> (Patients, Time, Features)
# We use .values to convert dataframe to numpy array
try:
    X_final = df[feature_cols].values.reshape(num_patients, num_hours, num_features)
    print(f"X_final Shape Created: {X_final.shape}")
except ValueError as e:
    print(f"Error reshaping: {e}")
    print("Check if every patient has exactly 24 rows. The previous script should have ensured this.")
    exit()

# --- Construct y (Labels) ---
# We must ensure y[i] corresponds exactly to X_final[i]
y_final = []
missing_labels = 0

for pid in unique_patients:
    try:
        # Look up the label in the admissions table
        label = admission_labels.loc[pid, 'hospital_expire_flag']
        y_final.append(label)
    except KeyError:
        # Should not happen if data is consistent, but good to handle
        print(f"Warning: Label not found for hadm_id {pid}")
        y_final.append(0) # Assume survival if missing (or exclude)
        missing_labels += 1

y_final = np.array(y_final)
print(f"y_final Shape Created: {y_final.shape}")

if missing_labels > 0:
    print(f"Warning: {missing_labels} patients were missing labels.")

# ==========================================
# 9. Inspect Sample Alignment
# ==========================================
print("\nStep 5: Verification...")
sample_idx = 0
print(f"Patient ID: {unique_patients[sample_idx]}")
print(f"Label (y): {y_final[sample_idx]} (0=Survive, 1=Die)")
print(f"Data Shape (X): {X_final[sample_idx].shape}")
print("Sample Data (First 3 hours of Heart Rate):")
# Assuming Heart Rate is the first column (index 0)
print(X_final[sample_idx, :3, 0]) 

# ==========================================
# 10. Save Final Datasets
# ==========================================
print("\nStep 6: Saving .npy Files...")

np.save(get_proc_path('X_final.npy'), X_final)
np.save(get_proc_path('y_final.npy'), y_final)

print("SUCCESS: X_final.npy and y_final.npy saved.")
print("You are now ready for Milestone 3: Model Training.")

Step 1: Loading Admissions Data...
Admissions Loaded. Total Records: 275
          hospital_expire_flag
hadm_id                       
24181354                     0
25926192                     0
23983182                     0
22942076                     1
21606243                     0

Step 2: Checking Class Balance...
Survived (0): 260
Expired  (1): 15
Imbalance Ratio: 1:17

Step 3: Loading Processed Time-Series Data...
Time-Series Shape: (5640, 19)

Step 4: Aligning X and y...
Dimensions -> Patients: 235, Hours: 24, Features: 17
X_final Shape Created: (235, 24, 17)
y_final Shape Created: (235,)

Step 5: Verification...
Patient ID: 20044587.0
Label (y): 0 (0=Survive, 1=Die)
Data Shape (X): (24, 17)
Sample Data (First 3 hours of Heart Rate):
[0. 0. 0.]

Step 6: Saving .npy Files...
SUCCESS: X_final.npy and y_final.npy saved.
You are now ready for Milestone 3: Model Training.
